In [0]:
/FileStore/tables/BankChurners.csv


In [0]:
df_sales = spark.read.csv("/FileStore/tables/sales_transaction06_12_2020_10_20_33.csv",header=True)


In [0]:
display(df_sales.limit(10))

PROD_ID,CUST_ID,TIME_ID,CHANNEL_ID,PROMO_ID,QUANTITY_SOLD,AMOUNT_SOLD
13,987,10-01-98,3,999,1,1232.16
13,1660,10-01-98,3,999,1,1232.16
13,1762,10-01-98,3,999,1,1232.16
13,1843,10-01-98,3,999,1,1232.16
13,1948,10-01-98,3,999,1,1232.16
13,2273,10-01-98,3,999,1,1232.16
13,2380,10-01-98,3,999,1,1232.16
13,2683,10-01-98,3,999,1,1232.16
13,2865,10-01-98,3,999,1,1232.16
13,4663,10-01-98,3,999,1,1232.16


In [0]:
from pyspark.sql.functions import concat,col,crc32,row_number
from pyspark.sql.window import Window
df_sales = df_sales.withColumn("id",concat(col("PROD_ID"),col("CUST_ID"),col("TIME_ID"),col("CHANNEL_ID")))
df_sales = df_sales.withColumn("crc32key",crc32(col("id")))
df_sales = df_sales.withColumn("duplicates",row_number().over(Window.partitionBy("crc32key").orderBy("crc32key")))
df_sales.createOrReplaceTempView("sales_v")

In [0]:
%sql
select * from sales_v where crc32key in (select crc32key from sales_v where duplicates>1)


PROD_ID,CUST_ID,TIME_ID,CHANNEL_ID,PROMO_ID,QUANTITY_SOLD,AMOUNT_SOLD,id,crc32key,duplicates
